In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
dtw = pd.read_csv('torem_dtw.csv')

In [9]:
dtw.head()

,ID,Nama Daya Tarik Wisata,Alamat,Daerah,Provinsi,Kawasan,Kategori,Google Review,Google Map ID
0,1,Pantai Kuta,"Kuta, Badung, Bali",Kuta,Bali,Pantai,Alam,4.4,ChIJkbJrRYFH0i0RggJb7CncdHE
1,2,Peninsula Island,"Pulau Peninsula Nusa Dua, Nusa Dua, Benoa, Kec...",Nusa Dua,Bali,Pantai,Alam,4.5,ChIJM51wq9dC0i0RktOawQYW3jk
2,3,Pura Luhur Uluwatu,"Pecatu, Kec. Kuta Sel., Kabupaten Badung, Bali",Pecatu,Bali,Pantai,Budaya,4.6,ChIJkYHLIPxP0i0RlQTbpxuNuQw
3,4,Garuda Wisnu Kencana,"Jl. Raya Uluwatu, Ungasan, Kuta Selatan Badung...",Ungasan,Bali,Kota,Monumen,4.5,ChIJx97hVM9E0i0RUVpPBj5miBk
4,5,Pantai Padang - Padang,"Pecatu, Kuta Selatan, Kabupaten Badung, Bali.",Pecatu,Bali,Pantai,Alam,4.4,ChIJV-JMXVBF0i0RFUBMWOw7eGQ


In [8]:
dtw.loc[dtw['Google Review'].isna(), 'Google Review'] = 4.4
dtw[dtw['Google Review'].isna()]

,ID,Nama Daya Tarik Wisata,Alamat,Daerah,Provinsi,Kawasan,Kategori,Google Review,Google Map ID


In [10]:
dtw.isna().sum()

ID                        0
Nama Daya Tarik Wisata    0
Alamat                    0
Daerah                    0
Provinsi                  0
Kawasan                   0
Kategori                  0
Google Review             0
Google Map ID             0
dtype: int64

In [46]:
kategori = pd.get_dummies(dtw[["Kategori"]])
kategori

,Kategori_Alam,Kategori_Budaya,Kategori_Monumen,Kategori_Rekreasi
0,1,0,0,0
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0
4,1,0,0,0
...,...,...,...,...
101,0,0,0,1
102,0,0,0,1
103,1,0,0,0
104,0,0,1,0


In [47]:
kawasan = pd.get_dummies(dtw[["Kawasan"]])
kawasan

,Kawasan_Air Terjun,Kawasan_Bukit,Kawasan_Danau,Kawasan_Desa,Kawasan_Gunung,Kawasan_Kota,Kawasan_Pantai,Kawasan_Taman
0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
101,0,0,0,0,0,1,0,0
102,0,0,0,0,0,1,0,0
103,0,0,0,0,0,0,1,0
104,0,0,0,0,0,1,0,0


In [48]:
provinsi = pd.get_dummies(dtw[["Provinsi"]])
provinsi

,Provinsi_Bali,Provinsi_Banten,Provinsi_DI Yogyakarta,Provinsi_Jakarta,Provinsi_Jawa Barat,Provinsi_Jawa Tengah,Provinsi_Jawa Timur,Provinsi_Nusa Tenggara Barat,Provinsi_Nusa Tenggara Timur
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
101,0,0,0,1,0,0,0,0,0
102,0,0,0,1,0,0,0,0,0
103,0,0,0,1,0,0,0,0,0
104,0,0,0,1,0,0,0,0,0


In [21]:
dtw["Google Review"] = dtw["Google Review"].astype(float)

In [49]:
dtw_features = pd.concat([provinsi, kawasan,kategori,dtw[['Google Review']]],axis=1)
dtw_features.head()

,Provinsi_Bali,Provinsi_Banten,Provinsi_DI Yogyakarta,Provinsi_Jakarta,Provinsi_Jawa Barat,Provinsi_Jawa Tengah,Provinsi_Jawa Timur,Provinsi_Nusa Tenggara Barat,Provinsi_Nusa Tenggara Timur,Kawasan_Air Terjun,...,Kawasan_Desa,Kawasan_Gunung,Kawasan_Kota,Kawasan_Pantai,Kawasan_Taman,Kategori_Alam,Kategori_Budaya,Kategori_Monumen,Kategori_Rekreasi,Google Review
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,4.4
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,4.5
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,4.6
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,4.5
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,4.4


In [16]:
from sklearn.preprocessing import MinMaxScaler

In [50]:
min_max_scaler = MinMaxScaler()
dtw_features = min_max_scaler.fit_transform(dtw_features)

In [51]:
np.round(dtw_features,2)

array([[1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.55],
       [1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.64],
       [1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.73],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.73],
       [0.  , 0.  , 0.  , ..., 1.  , 0.  , 0.73],
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.64]])

In [25]:
from sklearn.neighbors import NearestNeighbors

In [52]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(dtw_features)

In [53]:
distances, indices = nbrs.kneighbors(dtw_features)

In [73]:
def print_similar_dtw(id):
    print('Tempat yang mirip',dtw.at[id,'Nama Daya Tarik Wisata'], ':')
    for id in indices[id][1:]:
#         print(dtw.ix[id]["name"])
#         print(dtw[dtw['ID']==id])
        print(dtw.at[id,'Nama Daya Tarik Wisata'])


In [70]:
# df2[df2['user_id']==50]
# df.loc[df['Google Review'].isna(), 'Google Review'] = 4.4
# a= dtw.loc[dtw['ID']==1,'Nama Daya Tarik Wisata']
# indices
dtw.at[0,'Nama Daya Tarik Wisata']

'Pantai Kuta'

In [105]:
# type(indices)
# type(distances)
# list(indices)
# d = distances.tolist()
distances
# s=indices.tolist()

array([[0.        , 0.        , 0.09090909, 0.18181818, 1.41421356,
        1.41421356],
       [0.        , 0.09090909, 0.09090909, 0.09090909, 1.41421356,
        1.41421356],
       [0.        , 1.41421356, 1.41421356, 1.41713248, 1.41713248,
        1.41713248],
       [0.        , 1.41421356, 1.41421356, 1.41421356, 1.41421356,
        1.41713248],
       [0.        , 0.        , 0.09090909, 0.18181818, 1.41421356,
        1.41421356],
       [0.        , 0.09090909, 0.18181818, 0.18181818, 1.41421356,
        1.41421356],
       [0.        , 0.09090909, 0.18181818, 1.41713248, 1.41713248,
        1.41713248],
       [0.        , 1.41421356, 1.41421356, 1.41421356, 1.41421356,
        1.41713248],
       [0.        , 0.09090909, 0.18181818, 1.41421356, 1.41713248,
        1.44027087],
       [0.        , 1.41421356, 1.41421356, 1.41713248, 1.41713248,
        1.41713248],
       [0.        , 0.09090909, 0.09090909, 1.41421356, 1.41713248,
        1.42585338],
       [0.        , 1

In [88]:
def print_similar_dtw_w_list(id):
    print('Tempat yang mirip',dtw.at[id,'Nama Daya Tarik Wisata'], ':')
    for id in s[id][1:]:
#         print(dtw.ix[id]["name"])
#         print(dtw[dtw['ID']==id])
        print(dtw.at[id,'Nama Daya Tarik Wisata'])

In [107]:
print_similar_dtw_w_list(45)

Tempat yang mirip Gunung Api Purba Nglanggeran :
Gunung Semeru
Gunung Bromo
Gunung Bromo
Hutan Pinus Mangunan Dlingo
Puncak B29


In [79]:
print_similar_dtw(8)

Tempat yang mirip Desa Adat Penglipuran :
Tirta Empul 
Desa Trunyan
Wae Rebo
Pura Luhur Uluwatu
Desa Penujak


In [97]:
import csv 
Column = ['head', 'head', 'head', 'head', 'head', 'head']  
with open('indices.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(Column) 
    write.writerows(s) 

In [103]:
import csv 
Column = ['head', 'head', 'head', 'head', 'head', 'head']  
with open('distances.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(Column) 
    write.writerows(d) 

In [98]:
df = pd.read_csv('indices.csv')
df

,head,head.1,head.2,head.3,head.4,head.5
0,0,4,1,5,24,38
1,1,5,0,4,67,51
2,2,10,5,6,1,8
3,3,52,72,71,43,69
4,0,4,1,5,24,38
...,...,...,...,...,...,...
101,105,102,101,56,61,40
102,105,102,101,56,61,40
103,103,5,20,32,51,17
104,100,104,69,105,52,43


In [100]:
df.values.tolist() ==  s

True